# Segmenting and Clustering Neighbourhoods in Toronto

## About

This notebook was made for the IBM Capstone Project.

#### Import dependencies

In [1]:
#!pip install bs4
#!pip install requests
#!pip install html5lib

In [2]:
# import libraries
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np # library to handle data in a vectorized manner
from bs4 import BeautifulSoup # this module helps in web scrapping
import requests  # this module helps us to download a web page
import html5lib
import re  # for splitting using regex

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import pgeocode

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Part 1. Use Webscraping to create DataFrame on Toronto's postcodes, boroughs and neighbourhoods

#### Use Webscraping to extract Toronto neighbourhood Data

In [3]:
# this is the webpage url to extract the data from
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [4]:
# get the html data from the page
html_data = requests.get(url).text 

In [5]:
# convert all the html to soup
soup = BeautifulSoup(html_data,"html5lib")

In [6]:
# check html
soup

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en"><head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"c99b2258-4f15-4087-a5c3-417ec9515db4","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":1013111980,"wgRevisionId":1013111980,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","Wikipedia semi-protected pag

In [7]:
postcode_data = pd.DataFrame(columns=["col0", "col1", "col2", "col3", "col4", "col5", "col6", "col7", "col8"])

for row in soup.find("tbody").find_all("tr"):
    col = row.find_all("td")
    col0 = col[0].text
    col1 = col[1].text
    col2 = col[2].text
    col3 = col[3].text
    col4 = col[4].text
    col5 = col[5].text
    col6 = col[6].text
    col7 = col[7].text
    col8 = col[8].text
    
    postcode_data = postcode_data.append({"col0":col0, "col1":col1, "col2":col2, "col3":col3, "col4":col4, "col5":col5, "col6":col6,"col7":col7, "col8":col8}, ignore_index=True)

In [8]:
num_rows, num_cols = postcode_data.shape
num_cols

9

In [9]:
postcode_data

,col0,col1,col2,col3,col4,col5,col6,col7,col8
0,\nM1ANot assigned\n\n,\nM2ANot assigned\n\n,\nM3ANorth York(Parkwoods)\n\n,\nM4ANorth York(Victoria Village)\n\n,\nM5ADowntown Toronto(Regent Park / Harbourfro...,\nM6ANorth York(Lawrence Manor / Lawrence Heig...,\nM7AQueen's Park(Ontario Provincial Governmen...,\nM8ANot assigned\n\n,\nM9AEtobicoke(Islington Avenue)\n\n
1,\nM1BScarborough(Malvern / Rouge)\n\n,\nM2BNot assigned\n\n,\nM3BNorth York(Don Mills)North\n\n,\nM4BEast York(Parkview Hill / Woodbine Garden...,"\nM5BDowntown Toronto(Garden District, Ryerson...",\nM6BNorth York(Glencairn)\n\n,\nM7BNot assigned\n\n,\nM8BNot assigned\n\n,\nM9BEtobicoke(West Deane Park / Princess Gard...
2,\nM1CScarborough(Rouge Hill / Port Union / Hig...,\nM2CNot assigned\n\n,\nM3CNorth York(Don Mills)South(Flemingdon Par...,\nM4CEast York(Woodbine Heights)\n\n,\nM5CDowntown Toronto(St. James Town)\n\n,\nM6CYork(Humewood-Cedarvale)\n\n,\nM7CNot assigned\n\n,\nM8CNot assigned\n\n,\nM9CEtobicoke(Eringate / Bloordale Gardens / ...
3,\nM1EScarborough(Guildwood / Morningside / Wes...,\nM2ENot assigned\n\n,\nM3ENot assigned\n\n,\nM4EEast Toronto(The Beaches)\n\n,\nM5EDowntown Toronto(Berczy Park)\n\n,\nM6EYork(Caledonia-Fairbanks)\n\n,\nM7ENot assigned\n\n,\nM8ENot assigned\n\n,\nM9ENot assigned\n\n
4,\nM1GScarborough(Woburn)\n\n,\nM2GNot assigned\n\n,\nM3GNot assigned\n\n,\nM4GEast York(Leaside)\n\n,\nM5GDowntown Toronto(Central Bay Street)\n\n,\nM6GDowntown Toronto(Christie)\n\n,\nM7GNot assigned\n\n,\nM8GNot assigned\n\n,\nM9GNot assigned\n\n
5,\nM1HScarborough(Cedarbrae)\n\n,\nM2HNorth York(Hillcrest Village)\n\n,\nM3HNorth York(Bathurst Manor / Wilson Height...,\nM4HEast York(Thorncliffe Park)\n\n,\nM5HDowntown Toronto(Richmond / Adelaide / Ki...,\nM6HWest Toronto(Dufferin / Dovercourt Villag...,\nM7HNot assigned\n\n,\nM8HNot assigned\n\n,\nM9HNot assigned\n\n
6,\nM1JScarborough(Scarborough Village)\n\n,\nM2JNorth York(Fairview / Henry Farm / Oriole...,\nM3JNorth York(Northwood Park / York Universi...,\nM4JEast YorkEast Toronto(The Danforth East)...,\nM5JDowntown Toronto(Harbourfront East / Unio...,\nM6JWest Toronto(Little Portugal / Trinity)\n\n,\nM7JNot assigned\n\n,\nM8JNot assigned\n\n,\nM9JNot assigned\n\n
7,\nM1KScarborough(Kennedy Park / Ionview / East...,\nM2KNorth York(Bayview Village)\n\n,\nM3KNorth York(Downsview)East (CFB Toronto)\n\n,\nM4KEast Toronto(The Danforth West / Riverdal...,\nM5KDowntown Toronto(Toronto Dominion Centre ...,\nM6KWest Toronto(Brockton / Parkdale Village ...,\nM7KNot assigned\n\n,\nM8KNot assigned\n\n,\nM9KNot assigned\n\n
8,\nM1LScarborough(Golden Mile / Clairlea / Oakr...,\nM2LNorth York(York Mills / Silver Hills)\n\n,\nM3LNorth York(Downsview)West\n\n,\nM4LEast Toronto(India Bazaar / The Beaches W...,\nM5LDowntown Toronto(Commerce Court / Victori...,\nM6LNorth York(North Park / Maple Leaf Park /...,\nM7LNot assigned\n\n,\nM8LNot assigned\n\n,\nM9LNorth York(Humber Summit)\n\n
9,\nM1MScarborough(Cliffside / Cliffcrest / Scar...,\nM2MNorth York(Willowdale / Newtonbrook)\n\n,\nM3MNorth York(Downsview)Central\n\n,\nM4MEast Toronto(Studio District)\n\n,\nM5MNorth York(Bedford Park / Lawrence Manor ...,\nM6MYork(Del Ray / Mount Dennis / Keelsdale a...,\nM7MNot assigned\n\n,\nM8MNot assigned\n\n,\nM9MNorth York(Humberlea / Emery)\n\n


In [10]:
# declare variables
cell_value = ''
temp_postcode = ''
temp_borough = ''
part_before_backet_substring = ''
temp_postcodes = []
temp_boroughs = []
temp_neighbourhoods = []
split_point = 3  # num characters for postcode

# loop through each col
for col in range(0, num_cols):
    
    col_list = postcode_data[postcode_data.columns[col]]
    
    #col_list.map(lambda x: x.lstrip('\n').rstrip('\n\n'))
    
    # for each row in this column
    for row in range(0, num_rows):
        
        cell_value = col_list[row]
        # check to see if this postcode is assigned a neighbourhood, otherwise we don't want it
        if (cell_value.find('Not assigned')==-1):
            
            #if (cell_value.find('\n')==-1):
            cell_value = cell_value.replace('\n', ' ').strip()
            
            # get the part of the string before the bracket
            part_before_backet_substring = cell_value.split('(')[0]

            # get postcode from first 3 characters, get borough as the rest of the string
            temp_postcode, temp_borough = part_before_backet_substring[:split_point], part_before_backet_substring[split_point:]
            
            # check boroughs for boroughs that don't fit the pattern & fix them
            #if (temp_borough.find('East YorkEast Toronto')!=-1):
            #    temp_borough = temp_borough[:9] + '/' + temp_borough[9:]
            
            # get the substring inside the brackets
            bracket_substring = cell_value[cell_value.find("(")+1:cell_value.find(")")]
            # replace all '/' with ',' instead
            bracket_substring = bracket_substring.replace(' / ', ', ')

            temp_postcodes.append(temp_postcode)
            temp_boroughs.append(temp_borough)
            temp_neighbourhoods.append(bracket_substring)
                            

In [11]:
# merge the 3 lists into a single DataFrame
postcode_df = pd.DataFrame({'PostalCode': temp_postcodes, 'Borough': temp_boroughs, 'Neighbourhood': temp_neighbourhoods}, columns=['PostalCode', 'Borough', 'Neighbourhood'])
postcode_df.tail(25)

,PostalCode,Borough,Neighbourhood
78,M6K,West Toronto,"Brockton, Parkdale Village, Exhibition Place"
79,M6L,North York,"North Park, Maple Leaf Park, Upwood Park"
80,M6M,York,"Del Ray, Mount Dennis, Keelsdale and Silverthorn"
81,M6N,York,"Runnymede, The Junction North"
82,M6P,West Toronto,"High Park, The Junction South"
83,M6R,West Toronto,"Parkdale, Roncesvalles"
84,M6S,West Toronto,"Runnymede, Swansea"
85,M7A,Queen's Park,Ontario Provincial Government
86,M7R,MississaugaCanada Post Gateway Processing Centre,Enclave of L4W
87,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L


In [12]:
# fix up neighbourhood values that don't fit the usual pattern
postcode_df['Borough'] = postcode_df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                                         'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                                         'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                                         'MississaugaCanada Post Gateway Processing Centre':'Mississauga','The Danforth  East':'The Danforth East'})

In [13]:
postcode_df.tail(50)

,PostalCode,Borough,Neighbourhood
53,M5A,Downtown Toronto,"Regent Park, Harbourfront"
54,M5B,Downtown Toronto,"Garden District, Ryerson"
55,M5C,Downtown Toronto,St. James Town
56,M5E,Downtown Toronto,Berczy Park
57,M5G,Downtown Toronto,Central Bay Street
58,M5H,Downtown Toronto,"Richmond, Adelaide, King"
59,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands"
60,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange"
61,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel"
62,M5M,North York,"Bedford Park, Lawrence Manor East"


In [14]:
postcode_df.shape

(103, 3)

## Part 2. Get co-ordinates for the postcodes and add them to the DataFrame

In [15]:
temp_lats = []
temp_longs = []

# loop through postcode list looking up lat & log for each
for postal_code in temp_postcodes:
    nomi = pgeocode.Nominatim('ca')
    location = nomi.query_postal_code(postal_code)
    #print(location.latitude, location.longitude)
    # save lat & long to lists
    temp_lats.append(location.latitude)
    temp_longs.append(location.longitude)

In [16]:
# merge the 5 lists into a single DataFrame
df = pd.DataFrame({'PostalCode': temp_postcodes, 'Borough': temp_boroughs, 'Neighbourhood': temp_neighbourhoods, 'Latitude': temp_lats, 'Longitude': temp_longs}, columns=['PostalCode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude'])
df.tail(25)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
78,M6K,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.6383,-79.4301
79,M6L,North York,"North Park, Maple Leaf Park, Upwood Park",43.7137,-79.4869
80,M6M,York,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",43.6934,-79.4857
81,M6N,York,"Runnymede, The Junction North",43.6748,-79.4839
82,M6P,West Toronto,"High Park, The Junction South",43.6605,-79.4633
83,M6R,West Toronto,"Parkdale, Roncesvalles",43.6469,-79.4521
84,M6S,West Toronto,"Runnymede, Swansea",43.6512,-79.4828
85,M7A,Queen's Park,Ontario Provincial Government,43.6641,-79.3889
86,M7R,MississaugaCanada Post Gateway Processing Centre,Enclave of L4W,NaN,NaN
87,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,43.7804,-79.2505


In [17]:
df['Borough'] = df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                       'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                       'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                       'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [18]:
# check the new DataFrame with the co-ordinates
df.tail(25)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
78,M6K,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.6383,-79.4301
79,M6L,North York,"North Park, Maple Leaf Park, Upwood Park",43.7137,-79.4869
80,M6M,York,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",43.6934,-79.4857
81,M6N,York,"Runnymede, The Junction North",43.6748,-79.4839
82,M6P,West Toronto,"High Park, The Junction South",43.6605,-79.4633
83,M6R,West Toronto,"Parkdale, Roncesvalles",43.6469,-79.4521
84,M6S,West Toronto,"Runnymede, Swansea",43.6512,-79.4828
85,M7A,Queen's Park,Ontario Provincial Government,43.6641,-79.3889
86,M7R,Mississauga,Enclave of L4W,NaN,NaN
87,M7Y,East Toronto Business,Enclave of M4L,43.7804,-79.2505


In [19]:
# check how many postcodes are missing values
df.isna().sum()

PostalCode       0
Borough          0
Neighbourhood    0
Latitude         1
Longitude        1
dtype: int64

In [20]:
# drop rows if no co-ordinates can be looked up for that postcode
df = df.dropna(subset = ["Latitude"], inplace=False).reset_index(drop=True)

In [21]:
# check for missing values now
df.isna().sum()

PostalCode       0
Borough          0
Neighbourhood    0
Latitude         0
Longitude        0
dtype: int64

In [22]:
# use updated DataFrame to updated lists to use later
temp_postcodes = df['PostalCode']
temp_boroughs = df['Borough']
temp_neighbourhoods = df['Neighbourhood']
temp_lats = df['Latitude']
temp_longs = df['Longitude']

In [23]:
df.shape

(102, 5)

## Part 3. Visualize Toronto neighborhoods and show how they cluster together

#### Create a map of Toronto with neighbourhoods superimposed on top

In [24]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [25]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for postcode, lat, lng, borough, neighbourhood in zip(df['PostalCode'], df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {} {}'.format(neighbourhood, borough, postcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Use Foursquare API to get the top venues for each neighbourhood

In [26]:
CLIENT_ID = 'QX5S2PBXBBYJQFFFCTJUCYY0WCZC52WJGP4Z04LPWMH5PZ03' # your Foursquare ID
CLIENT_SECRET = 'WJMHCJY25OQOG1F34DZE1Y0RJWMGOH5EP5E052BYU5JESYGR' # your Foursquare Secret
ACCESS_TOKEN = 'RFLADLE0DBAIPYKWSZH0LFEKGIAGGMQSGLR5LKHEP203BUIG' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QX5S2PBXBBYJQFFFCTJUCYY0WCZC52WJGP4Z04LPWMH5PZ03
CLIENT_SECRET:WJMHCJY25OQOG1F34DZE1Y0RJWMGOH5EP5E052BYU5JESYGR


In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
toronto_venues = getNearbyVenues(temp_neighbourhoods, temp_lats, temp_longs)

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale
York Mills West
Willowdale
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
The Danforth  East
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawr

In [29]:
# check the size of the resulting dataframe
print(toronto_venues.shape)
toronto_venues.head()

(2153, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.8113,-79.1930,Canadian Appliance Source Whitby,43.808353,-79.191331,Home Service
1,"Guildwood, Morningside, West Hill",43.7678,-79.1866,Chick-N-Joy,43.768752,-79.187982,Fried Chicken Joint
2,"Guildwood, Morningside, West Hill",43.7678,-79.1866,Little Caesars Pizza,43.769046,-79.184386,Pizza Place
3,"Guildwood, Morningside, West Hill",43.7678,-79.1866,LCBO,43.771462,-79.184384,Liquor Store
4,"Guildwood, Morningside, West Hill",43.7678,-79.1866,Bulk Barn,43.771342,-79.184341,Food & Drink Shop


In [30]:
# check how many venues were returned for each neighbourhood
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Wilson Heights, Downsview North",7,7,7,7,7,7
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
Berczy Park,93,93,93,93,93,93
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",39,39,39,39,39,39
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",58,58,58,58,58,58


In [31]:
# find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 257 uniques categories.


### Analyze each neighbourhood

In [32]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Gym,College Rec Center,College Stadium,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,

In [33]:
# get new DataFrame size
toronto_onehot.shape

(2153, 258)

In [34]:
# group rows by neighbourhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Gym,College Rec Center,College Stadium,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.0000

In [35]:
# get size of new DataFrame
toronto_grouped.shape

(94, 258)

In [36]:
# print each neighbourhood along with the top 5 most common venues
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1            Badminton Court  0.25
2             Breakfast Spot  0.25
3         Chinese Restaurant  0.25
4              Moving Target  0.00


----Alderwood, Long Branch----
               venue  freq
0     Sandwich Place  0.12
1       Dance Studio  0.12
2        Pizza Place  0.12
3  Convenience Store  0.12
4        Coffee Shop  0.12


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                Pizza Place  0.14
1  Middle Eastern Restaurant  0.14
2                Coffee Shop  0.14
3              Deli / Bodega  0.14
4        Fried Chicken Joint  0.14


----Bayview Village----
                 venue  freq
0                 Park  0.25
1          Flower Shop  0.25
2                Trail  0.25
3          Gas Station  0.25
4  Moroccan Restaurant  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.08
1        

### Put top venues for each neighbourhood into a pandas dataframe

In [37]:
# function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [38]:
# create the new dataframe and display the top 10 venues for each neighbourhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.tail(50)

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,"Kensington Market, Chinatown, Grange Park",Café,Mexican Restaurant,Coffee Shop,Vegetarian / Vegan Restaurant,Park,Farmers Market,Gaming Cafe,Vietnamese Restaurant,Grocery Store,Tea Room
45,"Kingsview Village, St. Phillips, Martin Grove ...",Sandwich Place,Bank,Bus Line,Supermarket,Gas Station,Chinese Restaurant,Beer Store,Mobile Phone Shop,Pizza Place,Pharmacy
46,"Lawrence Manor, Lawrence Heights",Clothing Store,Coffee Shop,Women's Store,Restaurant,Sandwich Place,Cosmetics Shop,Toy / Game Store,Men's Store,Juice Bar,Sushi Restaurant
47,Lawrence Park,Park,Photography Studio,ATM,Music Venue,Music Store,Museum,Moving Target,Movie Theater,Moroccan Restaurant,Monument / Landmark
48,Leaside,Department Store,Electronics Store,Restaurant,Sporting Goods Shop,Sports Bar,Bank,Bakery,Pet Store,Sandwich Place,Coffee Shop
49,"Little Portugal, Trinity",Bar,Vegetarian / Vegan Restaurant,Cocktail Bar,Coffee Shop,Restaurant,Asian Restaurant,Vietnamese Restaurant,Pizza Place,Skating Rink,Seafood Restaurant
50,"Malvern, Rouge",Home Service,ATM,New American Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant
51,"Milliken, Agincourt North, Steeles East, L'Amo...",Intersection,Pharmacy,ATM,Movie Theater,Neighborhood,Music Venue,Music Store,Museum,Moving Target,Monument / Landmark
52,"Mimico NW, The Queensway West, South of Bloor,...",Burrito Place,Gym / Fitness Center,Sandwich Place,Food & Drink Shop,Middle Eastern Restaurant,Social Club,Burger Joint,Sushi Restaurant,Liquor Store,Thai Restaurant
53,"Moore Park, Summerhill East",Park,Playground,Grocery Store,Trail,Gym,Thai Restaurant,Moving Target,Music Venue,Music Store,Museum


###  Cluster neighbourhoods using K-means

In [39]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [40]:
toronto_grouped_clustering.head()

,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Gym,College Rec Center,College Stadium,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.000000,

#### Create a new dataframe that includes the cluster as well as the top 10 venues for each neighbourhood

In [41]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighbourhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), how='right', on='Neighbourhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,M1S,Scarborough,Agincourt,43.7946,-79.2644,1,Latin American Restaurant,Badminton Court,Breakfast Spot,Chinese Restaurant,Moving Target,Neighborhood,Music Venue,Music Store,Museum,ATM
88,M8W,Etobicoke,"Alderwood, Long Branch",43.6021,-79.5402,1,Sandwich Place,Dance Studio,Pizza Place,Convenience Store,Coffee Shop,Pub,Pharmacy,Gym,Museum,New American Restaurant
28,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.7535,-79.4472,1,Pizza Place,Middle Eastern Restaurant,Coffee Shop,Deli / Bodega,Fried Chicken Joint,Mediterranean Restaurant,Grocery Store,ATM,Moving Target,Music Store
19,M2K,North York,Bayview Village,43.7797,-79.3813,1,Park,Flower Shop,Trail,Gas Station,Moroccan Restaurant,Music Store,Museum,Moving Target,Movie Theater,ATM
62,M5M,North York,"Bedford Park, Lawrence Manor East",43.7335,-79.4177,1,Italian Restaurant,Coffee Shop,Sandwich Place,Indian Restaurant,Restaurant,Café,Butcher,Pizza Place,Breakfast Spot,Sushi Restaurant


### Visualise the clusters on the map

In [42]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Cluster 1

#### These neighbourhoods seem have these in common: Home Services, ATMs, Moroccan Restaurants

In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
95,North York,0,Home Service,Furniture / Home Store,ATM,Moroccan Restaurant,Music Venue,Music Store,Museum,Moving Target,Movie Theater,Molecular Gastronomy Restaurant
0,Scarborough,0,Home Service,ATM,New American Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant


#### Cluster 2

#### These neighbourhoods seem have a lot of parks, coffee shops, pizza places

In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Scarborough,1,Latin American Restaurant,Badminton Court,Breakfast Spot,Chinese Restaurant,Moving Target,Neighborhood,Music Venue,Music Store,Museum,ATM
88,Etobicoke,1,Sandwich Place,Dance Studio,Pizza Place,Convenience Store,Coffee Shop,Pub,Pharmacy,Gym,Museum,New American Restaurant
28,North York,1,Pizza Place,Middle Eastern Restaurant,Coffee Shop,Deli / Bodega,Fried Chicken Joint,Mediterranean Restaurant,Grocery Store,ATM,Moving Target,Music Store
19,North York,1,Park,Flower Shop,Trail,Gas Station,Moroccan Restaurant,Music Store,Museum,Moving Target,Movie Theater,ATM
62,North York,1,Italian Restaurant,Coffee Shop,Sandwich Place,Indian Restaurant,Restaurant,Café,Butcher,Pizza Place,Breakfast Spot,Sushi Restaurant
56,Downtown Toronto,1,Coffee Shop,Bakery,Hotel,Café,Seafood Restaurant,Restaurant,Japanese Restaurant,Cocktail Bar,Beer Bar,Farmers Market
9,Scarborough,1,Skating Rink,General Entertainment,College Stadium,Café,Movie Theater,Music Venue,Music Store,Museum,Moving Target,ATM
78,West Toronto,1,Café,Coffee Shop,Breakfast Spot,Thrift / Vintage Store,Gift Shop,Bar,Diner,Brewery,Supermarket,Chiropractor
68,Downtown Toronto,1,Coffee Shop,Italian Restaurant,Bar,Café,Bank,Restaurant,Bakery,Park,Gym / Fitness Center,Grocery Store
74,York,1,Park,Mexican Restaurant,Women's Store,Sporting Goods Shop,Bakery,Gym,Beer Store,Movie Theater,Music Store,Museum


#### Cluster 3

#### These neighbourhoods seem have a lot of parks, ATMs and either music venues/stores and or restaurants

In [45]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
82,West Toronto,2,Park,Intersection,Residential Building (Apartment / Condo),ATM,Music Venue,Music Store,Museum,Moving Target,Movie Theater,Moroccan Restaurant
17,North York,2,Park,Residential Building (Apartment / Condo),Music Venue,Music Store,Museum,Moving Target,Movie Theater,Moroccan Restaurant,ATM,New American Restaurant
44,Central Toronto,2,Park,Photography Studio,ATM,Music Venue,Music Store,Museum,Moving Target,Movie Theater,Moroccan Restaurant,Monument / Landmark
25,North York,2,Food & Drink Shop,Park,ATM,Neighborhood,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark
97,York,2,Park,ATM,New American Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant
23,North York,2,Park,Convenience Store,Daycare,ATM,Moving Target,Neighborhood,Music Venue,Music Store,Museum,Moroccan Restaurant


#### Cluster 4

#### This neighbourhood has a lot of auto garages, ATMs, and movie theaters

In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Scarborough,3,Auto Garage,ATM,Movie Theater,Neighborhood,Music Venue,Music Store,Museum,Moving Target,Moroccan Restaurant,Nightclub


#### Cluster 5

#### This neighbourhood has a lot of Korean or Mexican restaurants

In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,4,Korean BBQ Restaurant,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target
